Stabiliti care este sentimentul transmis prin mesajul By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly transportation, and I’m proud to be part of that movement..

Folositi:

clasificatorul bazat pe k-means antrenat de voi (folosind o biblioteca care implementeaza algoritmul k-means)

In [28]:
# Pasul 1 - incarcare date

import csv
import os
crtDir = os.getcwd()
fileName = os.path.join(crtDir, 'reviews_mixed.csv')

data = []
with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            dataNames = row
        else:
            data.append(row)
        line_count += 1

inputs = [data[i][0] for i in range(len(data))][:100]
outputs = [data[i][1] for i in range(len(data))][:100]
labelNames = list(set(outputs))

print(inputs[:2])
print("The label types are: ", labelNames[:2])

['The rooms are extremely small, practically only a bed.', 'Room safe did not work.']
The label types are:  ['positive', 'negative']


In [29]:
# Pasul 2 - impartire date (antrenament si test)

import numpy as np

np.random.seed(5)

noSamples = len(inputs)
indexes = [i for i in range(noSamples)]
trainSample = np.random.choice(indexes, int(0.8 * noSamples), replace=False)
testSample = [i for i in indexes if not i in trainSample]

trainInputs = [inputs[i] for i in trainSample]
trainOutputs = [outputs[i] for i in trainSample]
testInputs = [inputs[i] for i in testSample]
testOutputs = [outputs[i] for i in testSample]


In [30]:
# Pasul 3 - extragere caracteristici


from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=50)

trainFeatures = vectorizer.fit_transform(trainInputs)
testFeatures = vectorizer.transform(testInputs)


vectorizer = TfidfVectorizer(max_features=50)


def featureComputation(vectorizer, data):
    features = vectorizer.fit_transform(data).toarray()
    return features


trainFeatures = featureComputation(vectorizer, trainInputs)
testFeatures = featureComputation(vectorizer, testInputs)

print('Vocabulary: ', vectorizer.get_feature_names_out()[:10])
# print('Features: ', trainFeatures[:3])

Vocabulary:  ['aircon' 'and' 'are' 'bathroom' 'bed' 'building' 'cleaned' 'comfortable'
 'filthy' 'for']


In [31]:
#Pasul 4 - antrenare model de invatare nesupervizata (clustering)

from sklearn.cluster import KMeans
unsupervisedClassifier = KMeans(n_clusters=2, random_state=0)
unsupervisedClassifier.fit(trainFeatures)

KMeans(n_clusters=2, random_state=0)

In [32]:
#Pasul 5 - testare model
computedTestIndexes = unsupervisedClassifier.predict(testFeatures)
computedTestOutputs = [labelNames[value] for value in computedTestIndexes]
for i in range(0, len(testInputs)):
    print(testInputs[i], " -> ", computedTestOutputs[i])

The bed is very comfortable.  ->  negative
Very spacious rooms, quiet and very comfortable.  ->  negative
Corridors filthy
Room filthy
Electrical cables in room not safe
Whole building smelly
Shower repulsive  ->  negative
walls seem to have no sound insulation  ->  negative
The building was under renovation,  ->  negative
no elevator might be a challenge for some people  ->  positive
The bed was highly uncomfortable, although the engineer fixed it  ->  negative
bed, smell.  ->  negative
Detest the glass "door" if shower/tub .. with?  ->  negative
this was expected, clean towels and room cleaned every day.  ->  negative
More plug outlets with surge protectors.  ->  negative
Room was very spacious  ->  positive
Roof terrace great  ->  negative
No tea or coffee making facilities in the rooms  ->  negative
the room had aircon and we had earplugs and slept soundly.  ->  negative
Also, when the bright bathroom lights are turned on, it lights up the whole hotel room, shining thru the frosted

In [33]:
 #Pasul 6 - calcul metrici de performanta
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(testOutputs, computedTestOutputs))
print("\n")

Accuracy:  0.7




In [34]:
#folosim clasificatorul bazat pe k-means antrenat de noi pentru a clasifica textul dat
text =[ "By choosing a bike over a car, I’m reducing my environmental footprint. Cycling promotes eco-friendly "
        "transportation, and I’m proud to be part of that movement."]
text_features = vectorizer.transform(text)

# Predicția clusterului pentru mesaj
text_cluster_index = unsupervisedClassifier.predict(text_features)[0]
computedTestOutputs = labelNames[text_cluster_index]
print("Prediction for given input text -->", computedTestOutputs)

Prediction for given input text --> negative


In [35]:
from sklearn.metrics import accuracy_score
true_label= "positive"
print("acc: ", accuracy_score([true_label], [computedTestOutputs]))
print("\n")

acc:  0.0




EXTRAGERE CARACTERISTICI DIN TEXTE FOLOSIND ALTE CARACTERISTICI

In [9]:
def average_word_length(text):
    words = text.split()
    if len(words) == 0:
        return 0
    total_length = sum(len(word) for word in words)
    return total_length / len(words)

In [12]:
import nltk
import ssl
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
# 
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def extract_features(text):
    # Numărul de cuvinte
    word_count = len(word_tokenize(text))
    # Numărul de caractere
    char_count = len(text)
    # Numărul de propoziții
    sentence_count = len(sent_tokenize(text))
    # Tokenizare și etichetare POS (Part of Speech)
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    # Numărul de adjective și verbe
    adjective_count = len([word for word, tag in pos_tags if tag.startswith('JJ')])
    verb_count = len([word for word, tag in pos_tags if tag.startswith('VB')])

    return [word_count, char_count, sentence_count, adjective_count, verb_count]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [13]:
# Extrage caracteristici pentru seturile de antrenament și test

trainAvgWordLength = [average_word_length(text) for text in trainInputs]
testAvgWordLength = [average_word_length(text) for text in testInputs]

trainAvgWordLength = np.array(trainAvgWordLength).reshape(-1, 1)
testAvgWordLength = np.array(testAvgWordLength).reshape(-1, 1)

# Concatenam noile caracrcteristici celor vechi
trainFeaturesExtended = np.hstack((trainFeatures, trainAvgWordLength , np.array([extract_features(text) for text in trainInputs])))
testFeaturesExtended = np.hstack((testFeatures,testAvgWordLength, np.array([extract_features(text) for text in testInputs])))

In [14]:
# Antrenare
unsupervisedClassifier.fit(trainFeaturesExtended)

# Testare
computedTestIndexes = unsupervisedClassifier.predict(testFeaturesExtended)
computedTestOutputs = [labelNames[value] for value in computedTestIndexes]
for i in range(0, len(testInputs)):
    print(testInputs[i], " -> ", computedTestOutputs[i])

print("Accuracy after the new feature: ", accuracy_score(testOutputs, computedTestOutputs))
print("\n")

The bed is very comfortable.  ->  positive
Very spacious rooms, quiet and very comfortable.  ->  positive
Corridors filthy
Room filthy
Electrical cables in room not safe
Whole building smelly
Shower repulsive  ->  negative
walls seem to have no sound insulation  ->  positive
The building was under renovation,  ->  positive
no elevator might be a challenge for some people  ->  positive
The bed was highly uncomfortable, although the engineer fixed it  ->  positive
bed, smell.  ->  positive
Detest the glass "door" if shower/tub .. with?  ->  positive
this was expected, clean towels and room cleaned every day.  ->  positive
More plug outlets with surge protectors.  ->  positive
Room was very spacious  ->  positive
Roof terrace great  ->  positive
No tea or coffee making facilities in the rooms  ->  positive
the room had aircon and we had earplugs and slept soundly.  ->  positive
Also, when the bright bathroom lights are turned on, it lights up the whole hotel room, shining thru the frosted

ALTERNATIVE LA K-MEANS SI ANALIZA PERFORMANTELOR

In [85]:
# DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) este un algoritm de clusterizare care funcționează pe baza densității de puncte în spațiul de caracteristici. Iată o explicație teoretică a modului în care funcționează DBSCAN:

Puncte de bază (core points):
DBSCAN utilizează două parametri principale: epsilon (eps) și min_samples.
Un punct 
P
P este considerat un punct de bază (core point) dacă într-o sferă cu rază 
ϵ
ϵ (epsilon) centrată în jurul lui 
P
P există cel puțin \text{min_samples} puncte (inclusiv el însuși).
Vecini și puncte de frontieră:
Punctele care sunt în sfera de rază 
ϵ
ϵ a unui punct de bază, dar nu sunt ele însele puncte de bază, sunt denumite puncte de frontieră.
Un punct care nu este nici punct de bază, nici punct de frontieră este considerat zgomot (noise point).
Construirea clusterelor:
Algoritmul DBSCAN construiește clusterul pentru fiecare punct de bază și toate punctele de frontieră conectate la el (prin vecinii lor).
Dacă două puncte de bază sunt conectate direct sau prin alte puncte de bază, ele sunt parte din același cluster.
Detectarea zgomotului:
Punctele care nu sunt conectate la niciun alt punct sunt considerate zgomot și nu fac parte din niciun cluster.
Parametrii algoritmului:
Epsilon (eps) specifică raza maximă de distanță între două puncte pentru a fi considerate vecini.
min_samples specifică numărul minim de puncte necesare în sfera de rază epsilon pentru ca un punct să fie considerat punct de bază.

In [15]:
# DBSCAN

from sklearn.cluster import DBSCAN, AgglomerativeClustering

#min_samples = 5 indica cate data points minim sa fie intr o regiune pt a fi un cluster
#epsilon = 0.5 raza maxima de distanta dintre 2 pct pt a fi considerate grupate impreuna
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(trainFeatures)

# Testare performanta DBSCAN
computedTestIndexes = dbscan.fit_predict(testFeatures)
computedTestOutputs = [labelNames[value] for value in computedTestIndexes]

print("Accuracy DBSCAN: ", accuracy_score(testOutputs, computedTestOutputs))
print("\n")

Accuracy DBSCAN:  0.7




In [87]:
# AGLOMERATIV CLUSTERING

Agglomerative Clustering este un algoritm de tip bottom-up, care pornește de la fiecare punct într-un cluster separat și unește iterativ clusterele până când un anumit criteriu este îndeplinit. Iată o explicație teoretică a modului în care funcționează algoritmul Agglomerative Clustering:

Inițializare:
La început, fiecare punct de date este considerat un cluster individual.
Calcularea matricei de distanță:
Se calculează o matrice de distanță între toate perechile de puncte de date. Acesta poate fi, de exemplu, distanța Euclidiană sau orice altă măsură de similaritate între puncte.
Unirea celor mai apropiate clustere:
Se găsește cei doi clustere vecini cei mai apropiați și aceștia sunt uniți într-un nou cluster. Distanța între clustere poate fi măsurată în diverse moduri, cum ar fi distanța minimă, maximă sau media între punctele din cele două clustere.
Repetare:
Pasul 3 este repetat până când toate punctele sunt unite într-un singur cluster sau până când numărul dorit de clustere este atins.
Criterii de oprire:
Algoritmul se oprește când un anumit criteriu este îndeplinit, cum ar fi numărul dorit de clustere sau o anumită distanță maximă între clustere.

In [16]:
#Agglomerative Clustering
# #n_clusters = 2 indica cate clusteruri sunt create, deoarece clasificam datele in 'positive' si 'negative'

agglo = AgglomerativeClustering(n_clusters=2)
agglo_labels = agglo.fit_predict(trainFeatures)
computedTestIndexes = agglo.fit_predict(testFeatures)
computedTestOutputs = [labelNames[value] for value in computedTestIndexes]
print("Accuracy agglo: ", accuracy_score(testOutputs, computedTestOutputs))

Accuracy agglo:  0.45
